In [1]:
import pandas as pd

df = pd.read_parquet("uncertain_samples.parquet")

df

,filepath,pred_label,pred_conf
3546,data/imagenette/5/03546.jpg,French horn,0.106965
222,data/imagenette/2/00222.jpg,gas pump,0.107016
2080,data/imagenette/7/02080.jpg,French horn,0.107166
1096,data/imagenette/3/01096.jpg,French horn,0.107718
3167,data/imagenette/6/03167.jpg,English springer,0.108963
1631,data/imagenette/9/01631.jpg,chain saw,0.108974
804,data/imagenette/3/00804.jpg,tench,0.109512
2420,data/imagenette/1/02420.jpg,English springer,0.110322
1673,data/imagenette/9/01673.jpg,chain saw,0.110847
3638,data/imagenette/5/03638.jpg,gas pump,0.111175


In [2]:
train = pd.read_parquet("training_samples.parquet")
categories = train["label_name"].unique().tolist()


In [10]:
import gradio as gr

shortcut_js = """
<script>
function shortcuts(e) {
    // Only block shortcuts if we're in a text input or textarea
    if (e.target.tagName.toLowerCase() === "textarea" || 
        (e.target.tagName.toLowerCase() === "input" && e.target.type.toLowerCase() === "text")) {
        return;
    }
    
    if (e.key.toLowerCase() == "w") {
        document.getElementById("submit_btn").click();
    } else if (e.key.toLowerCase() == "d") {
        document.getElementById("next_btn").click();
    } else if (e.key.toLowerCase() == "a") {
        document.getElementById("back_btn").click();
    }
}
document.addEventListener('keypress', shortcuts, false);
</script>
"""

filepaths = df["filepath"].tolist()

with gr.Blocks(head=shortcut_js) as demo:
    current_index = gr.State(value=0)

    filename = gr.Textbox(
        label="Filename", value=filepaths[0], interactive=False
    )

    image = gr.Image(
        type="filepath", label="Image", value=filepaths[0], height=500
    )
    category = gr.Radio(choices=categories, label="Select Category")

    with gr.Row():
        back_btn = gr.Button("← Previous (A)", elem_id="back_btn")
        submit_btn = gr.Button(
            "Submit (W)",
            variant="primary",
            elem_id="submit_btn",
            interactive=False,
        )
        next_btn = gr.Button("Next → (D)", elem_id="next_btn")

    progress = gr.Slider(
        minimum=0,
        maximum=len(filepaths) - 1,
        value=0,
        label="Progress",
        interactive=False,
    )

    finish_btn = gr.Button("Finish Labeling", variant="primary")

    def update_submit_btn(choice):
        return gr.Button(interactive=choice is not None)

    category.change(
        fn=update_submit_btn, inputs=[category], outputs=[submit_btn]
    )

    def navigate(current_idx, direction):
        next_idx = current_idx + direction
        if 0 <= next_idx < len(filepaths):
            return filepaths[next_idx], filepaths[next_idx], next_idx, next_idx
        return (
            filepaths[current_idx],
            filepaths[current_idx],
            current_idx,
            current_idx,
        )

    def save_and_next(current_idx, selected_category):
        if selected_category is None:
            return (
                filepaths[current_idx],
                filepaths[current_idx],
                current_idx,
                current_idx,
            )

        # Save the current annotation
        with open("labeled.csv", "a") as f:
            f.write(f"{filepaths[current_idx]},{selected_category}\n")

        # Move to next image if not at the end
        next_idx = current_idx + 1
        if next_idx >= len(filepaths):
            return (
                filepaths[current_idx],
                filepaths[current_idx],
                current_idx,
                current_idx,
            )
        return filepaths[next_idx], filepaths[next_idx], next_idx, next_idx

    def convert_csv_to_parquet():
        try:
            df = pd.read_csv("labeled.csv", header=None)
            df.columns = ["filepath", "label"]
            df = df.drop_duplicates(subset=["filepath"], keep="last")
            df.to_parquet("labeled.parquet")
        except Exception as e:
            print(e)
            return

    back_btn.click(
        fn=lambda idx: navigate(idx, -1),
        inputs=[current_index],
        outputs=[filename, image, current_index, progress],
    )

    next_btn.click(
        fn=lambda idx: navigate(idx, 1),
        inputs=[current_index],
        outputs=[filename, image, current_index, progress],
    )

    submit_btn.click(
        fn=save_and_next,
        inputs=[current_index, category],
        outputs=[filename, image, current_index, progress],
    )

    finish_btn.click(fn=convert_csv_to_parquet)

demo.launch(height=1000)


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
